# Stars: 

### Day 20: Pulse Propagation

#### Part 1

In [6]:
def get_data(file_name):
    with open(file_name, "r") as file:
        lines = file.read() 
    lines = lines.split("\n")
    # lines = [section.split("\n") for section in sections]
    return lines

file_name = "test_input.txt"
config = get_data(file_name)
print(config)

['broadcaster -> a, b, c', '%a -> b', '%b -> c', '%c -> inv', '&inv -> a']


In [ ]:
# flip flop modules:
#   > prefix %
#   > either on or off
#   > initally off
#   > if recieves a high pulse, it does nothing
#   > if recieves a low pulse, it switches on/off
#       > if initally off, it turns on and sends a high pulse
#       > if initally on, it turns off and sends a low pulse

# conjuntion modules:
#   > prefix &
#   > remembers most recent pulse recieved from each of their connected input modules
#   > inital defaults to low pulse for each input
#   > when pulse is recieved
#       > first updates memory for that input
#       > if it remembers high pulses for all inputs, it sends a low pulse
#       > otherwise, it sends a high pulse

# broadcast modules:
#   > only one of these
#   > when recieved a pulse, sends the same pulse to all destination modules

# button modules:
#   > only one of these
#   > when recieved a pulse, sends the same pulse to all destination modules


In [75]:
class Module:
    def __init__(self, name):
        self.name = name
        self.destinations = []
        self.high_pulse_count = 0
        self.low_pulse_count = 0

    def add_destination(self, module):
        self.destinations.append(module)

    def send_pulse(self, pulse):
        for destination in self.destinations:
            destination.receive_pulse(pulse)
            
    def receive_pulse(self, pulse):
        if pulse == "high":
            self.high_pulse_count += 1
        elif pulse == "low":
            self.low_pulse_count += 1

In [76]:
class FlipFlop(Module):
    def __init__(self, name):
        super().__init__(name)
        self.state = False  # False for off, True for on

    def receive_pulse(self, pulse):
        if pulse == "low":
            self.state = not self.state
            if self.state:
                self.send_pulse("high")
            else:
                self.send_pulse("low")
        super().receive_pulse(pulse)

In [77]:
class Conjunction(Module):
    def __init__(self, name, num_inputs):
        super().__init__(name)
        self.inputs = {i: "low" for i in range(num_inputs)}  # Initialize input memories to low

    def receive_pulse(self, pulse):
        # Update input memory
        # Assumes pulse format: ("module_name", "pulse_type")
        module_name, pulse_type = pulse
        input_index = int(module_name.split("_")[-1])  # Extract input index from module name
        self.inputs[input_index] = pulse_type

        # Check if all inputs are high
        all_high = all(pulse_type == "high" for pulse_type in self.inputs.values())

        # Send pulse based on input status
        if all_high:
            self.send_pulse("low")
        else:
            self.send_pulse("high")
        super().receive_pulse(pulse)

In [78]:
class Broadcaster(Module):
    def receive_pulse(self, pulse):
        self.send_pulse(pulse)
        super().receive_pulse(pulse)

In [79]:
a = FlipFlop("%a")
b = FlipFlop("%b")
c = FlipFlop("%c")
inv = Conjunction("&inv", num_inputs=1)
broadcaster = Broadcaster("broadcaster")

broadcaster.add_destination(a)
broadcaster.add_destination(b)
broadcaster.add_destination(c)
a.add_destination(b)
b.add_destination(c)
c.add_destination(inv)
inv.add_destination(a)


In [80]:
broadcaster.receive_pulse("high")

In [81]:
a.receive_pulse("high")